First we will initialize our filiBERTo tokenizer like before, and use it to encode our data.

In [1]:
from transformers import RobertaTokenizer
import torch
from pathlib import Path
from tqdm.auto import tqdm
import random
import os

In [2]:
# initialize the tokenizer using the tokenizer we initialized and saved to file
tokenizer = RobertaTokenizer.from_pretrained('filiberto', max_len=512)

In [3]:
def mlm(tensor):
    # create random array of floats with equal dims to tensor
    rand = torch.rand(tensor.shape)
    # mask random 15% where token is not 0 <s>, 1 <pad>, or 2 <s/>
    mask_arr = (rand < .15) * (tensor != 0) * (tensor != 1) * (tensor != 2)
    # loop through each row in tensor (cannot do in parallel)
    for i in range(tensor.shape[0]):
        # get indices of mask positions from mask array
        selection = torch.flatten(mask_arr[i].nonzero()).tolist()
        # mask tensor
        tensor[i, selection] = 4
    return tensor

In [4]:
paths = [str(x) for x in Path('../../data/text/oscar_it').glob('**/*.txt')]
# initialize lists of tensors
input_ids = []
mask = []
labels = []
# open all files, encode and add to single dataset
for path in tqdm(paths[:200]):
    # :50
    # open the file and split into list by newline characters
    with open(path, 'r', encoding='utf-8') as fp:
        lines = fp.read().split('\n')
    # encode
    sample = tokenizer(lines, max_length=512, truncation=True, padding='max_length')
    # convert tokens to tensor
    labels.append(torch.tensor(sample.input_ids))
    # create attention mask tensor
    mask.append(torch.tensor(sample.attention_mask))
    # mask ~15% of tokens to create inputs
    input_ids.append(mlm(labels[-1].detach().clone()))
# convert lists of tensors into tensors
input_ids = torch.cat(input_ids)
mask = torch.cat(mask)
labels = torch.cat(labels)

100%|██████████| 200/200 [5:16:30<00:00, 94.95s/it]


We have 2000000 tokenized sequences, each containing 512 tokens:

In [5]:
input_ids.shape

torch.Size([2000000, 512])

In [6]:
input_ids[0]

tensor([    0,   696, 17974,  1298,  7479,   292,  1081,     4, 11222,  7427,
          787,   742,  2014,   280,    11, 10012,  3686,  1314,  2192,  1616,
          605,  1168, 14686,    30,   567,   267,  1320,    16,   383,  3439,
          458, 10418,  9303,   375, 23989,  2982,  1185,   404,  2506,   425,
        26047, 20159,   722,   304,   306, 22536,   375,  8832, 19775,   964,
          439,     4,  1466,   275,  2473,   404,    37,     4,     4,   722,
        15001,   272,   275,     4,   625,   771, 18443,   481,     4,   275,
        11936,   536,    18,   683,  3283, 22100,   375,     4, 15637,     4,
         1122,     4,   269,   427,   286, 12010,   481,  8605,  6033,   275,
         4858,    18, 11510,   368,   275,  6231,   315,     4,   582,  2215,
        12673,    18,     4,     4,   369,   451,  2147,     4,     4,   490,
          315,  6907,  6887,     4,   582,  2215, 12673,     4, 14091,     4,
          792, 10761,  6561,  2556,  5929,     4,   460,     4, 

We can see the special tokens here, `0` is our **<s\>** token, `2` our **<s\\>** token, `3` our **<mask\>** token, and at the end we have two `1` - or **<pad\>** - tokens.

Let's save these tensors to file for if we need to do any further training later.

In [7]:
import os

if not os.path.exists('./filiberto_training'):
    os.mkdir('./filiberto_training')

torch.save(input_ids, './filiberto_training/input_ids.pt')
torch.save(mask, './filiberto_training/attention_mask.pt')
torch.save(labels, './filiberto_training/labels.pt')

del input_ids, mask, labels

In [3]:
input_ids = torch.load('./filiberto_training/input_ids.pt')
mask = torch.load('./filiberto_training/attention_mask.pt')
labels = torch.load('./filiberto_training/labels.pt')

Now let's move onto building our dataset and dataloader.

In [4]:
encodings = {'input_ids': input_ids, 'attention_mask': mask, 'labels': labels}

In [5]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        # store encodings internally
        self.encodings = encodings

    def __len__(self):
        # return the number of samples
        return self.encodings['input_ids'].shape[0]

    def __getitem__(self, i):
        # return dictionary of input_ids, attention_mask, and labels for index i
        return {key: tensor[i] for key, tensor in self.encodings.items()}

Next we initialize our `Dataset`.

In [6]:
dataset = Dataset(encodings)

And initialize the dataloader, which will load the data into the model during training.

In [7]:
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

And move onto building our model, we first need to create a RoBERTa config object, which will describe which features we want to initialize our RoBERTa model with.

In [8]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=30_522,  # we align this to the tokenizer vocab set in previous notebook
    max_position_embeddings=514,
    hidden_size=768,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1
)

Then we import and initialize a RoBERTa model with a language modeling head.

In [9]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config)

And now we move onto training. First we setup GPU/CPU usage.

In [10]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
model.to(device)

Activate the training mode of our model, and initialize our optimizer (Adam with weighted decay - reduces chance of overfitting).

In [12]:
from transformers import AdamW

# activate training mode
model.train()
# initialize optimizer
optim = AdamW(model.parameters(), lr=1e-5)

In [14]:
# optionally, if using tensorboard, initialize writer object
writer = torch.utils.tensorboard.SummaryWriter()

Now we move onto the training loop.

In [15]:
from tqdm import tqdm  # for our progress bar

epochs = 1  # trained for 4 in total
step = 0

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # take loss for tensorboard
        writer.add_scalar('Loss/train', loss, step)
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())
        step += 1

Epoch 0: 100%|██████████| 125000/125000 [17:26:15<00:00,  1.99it/s, loss=0.489]


In [16]:
model.save_pretrained('./filiberto')